In [49]:
import pandas as pd
import numpy as np
import os
import keras
from datetime import timedelta, datetime

In [208]:
#os.chdir("src") #used to reset to original working directory

FileNotFoundError: [Errno 2] No such file or directory: 'src'

In [3]:
os.chdir("../") #Used to go out into the cmpe-351-group-1 folder to access the data without changing directories multiple times

In [324]:
def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
            +timedelta(hours=t.minute//30))

In [325]:
def normalize(df,column):
    m = df[column].mean()
    s = df[column].std()
    df[column] = df[column].apply(lambda x: (x-m)/s)
    return df[column]


In [326]:
##Dates is an array of tuples consisting of month and day as numbers
def get_dataset(crypto,start_month,start_day,end_month,end_day):
    twitter_df = pd.read_csv(f'data/processed/twitter/{crypto}_sentiment.csv')
    try:
        crypt_prices = pd.read_csv(f'data/raw/crypto/{crypto_dict[crypto]}_2022_{start_month}_{start_day}-2022_{end_month}_{end_day}_minute.csv')
    except:
        crypt_prices = pd.read_csv(f'data/raw/crypto/{crypto_dict[crypto]}_2022_{start_month}_{start_day}-2022_{end_month}_{end_day}.csv')
    crypt_prices["time"] = pd.to_datetime(crypt_prices['time'],unit='s')
    price_dict = crypt_prices.set_index("time")["open"].to_dict()
    twitter_df["created_at"] = twitter_df["created_at"].apply(lambda x: pd.to_datetime(x).tz_localize(None))
    twitter_df["created_at"] = twitter_df["created_at"].apply(lambda x: hour_rounder(x))
    twitter_df["future_date"] = twitter_df["created_at"].apply(lambda x: x+timedelta(hours=23))

    twitter_df = twitter_df[twitter_df["future_date"].isin(crypt_prices["time"])] ##Filter out dates for which we don't have price data

    #print(len(twitter_df))

    twitter_df = twitter_df[twitter_df["created_at"].isin(crypt_prices["time"])] ##Filter out dates for which we don't have price data


    twitter_df["price"] = twitter_df["created_at"].apply(lambda x: price_dict[x])
    for i in range(1,24):
        twitter_df[f"price_{i}hours"] = twitter_df["created_at"].apply(lambda x: price_dict[x+timedelta(hours=i)])
    twitter_df["retweet_count"] = normalize(twitter_df,"retweet_count")
    twitter_df["favourite_count"] = normalize(twitter_df,"retweet_count")
    twitter_df["followers_count"] = normalize(twitter_df,"retweet_count")
    twitter_df["authority"] = twitter_df[["retweet_count","favourite_count","followers_count"]].sum(axis=1)

    X = twitter_df[["authority","vader_sentiment_compound","price"]]
    y = twitter_df[[f"price_{i}hours" for i in range(1,24)]]
    return twitter_df,X,y


In [327]:
def make_model():
    model = keras.Sequential()
    model.add(keras.layers.LSTM(8,input_shape= (3,1)))
    model.add(keras.layers.Dense(32))
    model.add(keras.layers.Dense(23))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
    return model

In [328]:
def estimate_price(model,date,df):
    date_23_hours = date-timedelta(hours=23)
    date_1_hours = date-timedelta(hours=1)
    df_useful = df[(df["created_at"]<=date_1_hours) & (df["created_at"]>=date_23_hours)]
    if len(df_useful)==0:
        return ""
    else:
        df_useful["idx"] = df_useful["created_at"].apply(lambda x: round((date-x).seconds/60/60))
        X = df_useful[["authority","vader_sentiment_compound","price"]]
        y_hat = model.predict_on_batch(X)
        #print(y_hat)
        idx = df_useful["idx"].values
        vals = []
        for i in range(len(y_hat)):
            vals.append(y_hat[i][idx[i]-1])
        return np.mean(vals)

In [329]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)*24):
        yield start_date + timedelta(hours=n)

In [330]:
def make_prediction(start_date, end_date, model,df):
    pred = {}
    for date in daterange(datetime.strptime(start_date,"%d/%m/%Y"),datetime.strptime(end_date,"%d/%m/%Y")):
        val = estimate_price(model,date,df)
        if val!='':
            pred[date.strftime("%Y-%m-%d %H:%M:%S")] = val
    return pred

In [331]:
def evaluate_model(start_date,end_date,model,df_test):
    y_hat = make_prediction(start_date,end_date,model,df_test)
    df_temp = df_test.set_index("created_at")["price"].drop_duplicates()
    mse = 0
    for i in y_hat:
        try:
            mse+=(df_temp.loc[i].values[0]-y_hat[i])**2
        except:
            pass
    return mse/len(y_hat)



In [332]:
def make_time_series_dataset(crypto,start_month,start_day,end_month,end_day):
    X = []
    y = []
    try:
        crypt_prices = pd.read_csv(f'data/raw/crypto/{crypto_dict[crypto]}_2022_{start_month}_{start_day}-2022_{end_month}_{end_day}_minute.csv')

    except:
        crypt_prices = pd.read_csv(f'data/raw/crypto/{crypto_dict[crypto]}_2022_{start_month}_{start_day}-2022_{end_month}_{end_day}.csv')

    crypt_prices["time"] = pd.to_datetime(crypt_prices['time'],unit='s')

    crypt_prices_useful = {}

    for i in crypt_prices[["time","open"]].values:
        if int(i[0].strftime("%M"))==0 and int(i[0].strftime("%S"))==0:
            crypt_prices_useful[i[0]] = i[1]
    for i in crypt_prices_useful:
        if (i-timedelta(hours=23)) in crypt_prices_useful:
            X.append([crypt_prices_useful[i] for i in pd.date_range(i-timedelta(hours=23),i-timedelta(hours=1),freq="H")])
            y.append(crypt_prices_useful[i])
    return X,y


In [333]:
def make_ts_model():
    model = keras.Sequential()
    model.add(keras.layers.LSTM(8,input_shape= (23,1)))
    model.add(keras.layers.Dense(32))
    model.add(keras.layers.Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
    return model

In [334]:
df_avax,X_avax,y_avax = get_dataset("avalanche","03","05","03","11")
df_avax2, X_avax2, y_avax2 = get_dataset("avalanche","03","28","04","04")
df_avax3, X_avax3, y_avax3 = get_dataset("avalanche","03","11","03","12")
df_avax4, X_avax4, y_avax4 = get_dataset("avalanche","04","04","04","05")

df_avax = df_avax.append(df_avax2)
df_avax = df_avax.append(df_avax3)
df_avax = df_avax.append(df_avax4)

X_avax = X_avax.append(X_avax2)
X_avax = X_avax.append(X_avax3)
X_avax = X_avax.append(X_avax4)

y_avax = y_avax.append(y_avax2)
y_avax = y_avax.append(y_avax3)
y_avax = y_avax.append(y_avax4)

df_avax_test, _,_ = get_dataset("avalanche","04","05","04","07")


In [335]:
df_btc,X_btc,y_btc = get_dataset("bitcoin","03","05","03","11")
df_btc2, X_btc2, y_btc2 = get_dataset("bitcoin","03","28","04","04")
df_btc3, X_btc3, y_btc3 = get_dataset("bitcoin","03","11","03","12")
df_btc4, X_btc4, y_btc4 = get_dataset("bitcoin","04","04","04","05")

df_btc = df_btc.append(df_btc2)
df_btc = df_btc.append(df_btc3)
df_btc = df_btc.append(df_btc4)

X_btc = X_btc.append(X_btc2)
X_btc = X_btc.append(X_btc3)
X_btc = X_btc.append(X_btc4)

y_btc = y_btc.append(y_btc2)
y_btc = y_btc.append(y_btc3)
y_btc = y_btc.append(y_btc4)
df_btc_test, _,_ = get_dataset("bitcoin","04","05","04","07")


In [336]:
df_eth,X_eth,y_eth = get_dataset("ethereum","03","05","03","11")
df_eth2, X_eth2, y_eth2 = get_dataset("ethereum","03","28","04","04")
df_eth3,X_eth3,y_eth3 = get_dataset("ethereum","03","11","03","12")
df_eth4, X_eth4, y_eth4 = get_dataset("ethereum","04","04","04","05")

df_eth = df_eth.append(df_eth2)
df_eth = df_eth.append(df_eth3)
df_eth = df_eth.append(df_eth4)



X_eth = X_eth.append(X_eth2)
X_eth = X_eth.append(X_eth3)
X_eth = X_eth.append(X_eth4)

y_eth = y_eth.append(y_eth2)
y_eth = y_eth.append(y_eth3)
y_eth = y_eth.append(y_eth4)

df_eth_test, _,_ = get_dataset("ethereum","04","05","04","07")

In [337]:
df_sol,X_sol,y_sol = get_dataset("solana","03","05","03","11")
df_sol2, X_sol2, y_sol2 = get_dataset("solana","03","28","04","04")
df_sol3,X_sol3,y_sol3 = get_dataset("solana","03","11","03","12")
df_sol4, X_sol4, y_sol4 = get_dataset("solana","04","04","04","05")

df_sol = df_sol.append(df_sol2)
df_sol = df_sol.append(df_sol3)
df_sol = df_sol.append(df_sol4)


X_sol = X_sol.append(X_sol2)
X_sol = X_sol.append(X_sol3)
X_sol = X_sol.append(X_sol4)

y_sol = y_sol.append(y_sol2)
y_sol = y_sol.append(y_sol3)
y_sol = y_sol.append(y_sol4)

df_sol_test, _,_ = get_dataset("solana","04","05","04","07")

In [338]:
df_doge,X_doge,y_doge = get_dataset("doge","03","05","03","11")
df_doge2, X_doge2, y_doge2 = get_dataset("doge","03","28","04","04")
df_doge3,X_doge3,y_doge3 = get_dataset("doge","03","11","03","12")
df_doge4, X_doge4, y_doge4 = get_dataset("doge","04","04","04","05")

df_doge = df_doge.append(df_doge2)
df_doge = df_doge.append(df_doge3)
df_doge = df_doge.append(df_doge4)


X_doge = X_doge.append(X_doge2)
X_doge = X_doge.append(X_doge3)
X_doge = X_doge.append(X_doge4)

y_doge = y_doge.append(y_doge2)
y_doge = y_doge.append(y_doge3)
y_doge = y_doge.append(y_doge4)

df_doge_test, _,_ = get_dataset("solana","04","05","04","07")

In [ ]:
bit_model = make_model()
bit_model.fit(X_btc,y_btc,epochs=500,batch_size=10)

Epoch 1/500
163/163 [==============================] - 2s 3ms/step - loss: 2023683328.0000 - mean_squared_error: 2023683328.0000
Epoch 2/500
163/163 [==============================] - 0s 3ms/step - loss: 2022702080.0000 - mean_squared_error: 2022702080.0000
Epoch 3/500
163/163 [==============================] - 0s 3ms/step - loss: 2020833920.0000 - mean_squared_error: 2020833920.0000
Epoch 4/500
163/163 [==============================] - 0s 3ms/step - loss: 2017039616.0000 - mean_squared_error: 2017039616.0000
Epoch 5/500
163/163 [==============================] - 1s 3ms/step - loss: 2011054464.0000 - mean_squared_error: 2011054464.0000
Epoch 6/500
163/163 [==============================] - 0s 3ms/step - loss: 2002865664.0000 - mean_squared_error: 2002865664.0000
Epoch 7/500
163/163 [==============================] - 0s 3ms/step - loss: 1993021952.0000 - mean_squared_error: 1993021952.0000
Epoch 8/500
163/163 [==============================] - 0s 3ms/step - loss: 1981782784.0000 - mean

In [ ]:
eth_model = make_model()
eth_model.fit(X_eth,y_eth,epochs=1000,batch_size=10)

In [ ]:
doge_model = make_model()
doge_model.fit(X_doge,y_doge,epochs=1000,batch_size=10)

In [ ]:
sol_model = make_model()
sol_model.fit(X_sol,y_sol,epochs=1000,batch_size=10)

In [ ]:
avax_model = make_model()
avax_model.fit(X_avax,y_avax,epochs=1000,batch_size=10)

In [ ]:
evaluate_model("05/04/2022","07/04/2022",bit_model,df_btc_test)

In [ ]:
X_ts_btc ,y_ts_btc = make_time_series_dataset("bitcoin","01","03","04","03")
X_test_btc, y_test_btc = make_time_series_dataset("bitcoin","04","05","04","07")
bit_ltsm = make_ts_model()
bit_ltsm.fit(X_ts_btc,y_ts_btc,epochs=500,batch_size=10,validation_data=(X_test_btc,y_test_btc))

In [ ]:
X_ts_eth ,y_ts_eth = make_time_series_dataset("ethereum","01","03","04","03")
X_test_eth, y_test_eth = make_time_series_dataset("ethereum","04","05","04","07")
eth_ltsm = make_ts_model()
eth_ltsm.fit(X_ts_eth,y_ts_eth,epochs=500,batch_size=10)

In [ ]:
X_ts_avax ,y_ts_avax = make_time_series_dataset("avalanche","01","03","04","03")
X_test_avax, y_test_avax = make_time_series_dataset("avalanche","04","05","04","07")

avax_ltsm = make_ts_model()
avax_ltsm.fit(X_ts_avax,y_ts_avax,epochs=500,batch_size=10)

In [ ]:
X_ts_sol ,y_ts_sol = make_time_series_dataset("solana","01","03","04","03")
X_test_sol, y_test_sol = make_time_series_dataset("solana","04","05","04","07")

sol_ltsm = make_ts_model()
sol_ltsm.fit(X_ts_sol,y_ts_sol,epochs=500,batch_size=10)

In [ ]:
X_ts_doge ,y_ts_doge = make_time_series_dataset("doge","01","03","04","03")
X_test_doge, y_test_doge = make_time_series_dataset("doge","04","05","04","07")

doge_ltsm = make_ts_model()
doge_ltsm.fit(X_ts_doge,y_ts_doge,epochs=500,batch_size=10)

Summary:
